## Importing libraries/modules

In [1]:
import utils
import numpy as np
import pandas as pd
import os

In [2]:
DATA_DIR = 'Data'
CACHE_DIR = 'Cache'
SPEAKER_INFO_FILE_PATH = os.path.join(DATA_DIR, 'speaker_attributes.parquet')
CACHE_FILE_PATH = os.path.join(CACHE_DIR, 'processed_data.json.bz2')

In [3]:
import os
import time
import bz2
import json

In [ ]:
from sklearn.decomposition import TruncatedSVD

# /!\ TfidfVectorizer can't be batched! If appropriate, use CountVectorizer + TfidfTransformer instead
# (equivalent, but CountVectorizer can be batched).
@utils.cache_to_file_pickle("function-embed_sentences")
def embed_sentences(data_dir):
    count_vectorizer = TfidfVectorizer(strip_accents = "ascii", lowercase = True, stop_words = "english",
                                       norm = 'l2', use_idf = True, smooth_idf = True, sublinear_tf = False)
    
    matrix = count_vectorizer.fit_transform(line['quotation'] for line in all_quotes_generator(data_dir))
    
    return matrix


        
        
matrix = embed_sentences(data_dir = DATA_DIR)
matrix

Starting processing Data\quotes-2015.json.bz2
Processed 1000000 lines from Data\quotes-2015.json.bz2 in 1.471 minutes
Processed 2000000 lines from Data\quotes-2015.json.bz2 in 3.039 minutes
Processed 3000000 lines from Data\quotes-2015.json.bz2 in 4.674 minutes
Processed 4000000 lines from Data\quotes-2015.json.bz2 in 6.211 minutes
Processed 5000000 lines from Data\quotes-2015.json.bz2 in 7.806 minutes
Processed 6000000 lines from Data\quotes-2015.json.bz2 in 9.312 minutes
Processed 7000000 lines from Data\quotes-2015.json.bz2 in 10.867 minutes
Processed 8000000 lines from Data\quotes-2015.json.bz2 in 12.362 minutes
Processed 9000000 lines from Data\quotes-2015.json.bz2 in 14.016 minutes
Processed 10000000 lines from Data\quotes-2015.json.bz2 in 15.655 minutes
Processed 11000000 lines from Data\quotes-2015.json.bz2 in 17.365 minutes
Processed 12000000 lines from Data\quotes-2015.json.bz2 in 19.032 minutes
Processed 13000000 lines from Data\quotes-2015.json.bz2 in 20.582 minutes
Process

In [ ]:
from sklearn.decomposition import TruncatedSVD

TruncatedSVD(n_components = 100, n_iter = 10)